In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

In [2]:

import monai
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Scrolls_Dataset_Module import Scrolls_Dataset
import matplotlib.patches as patches
from lit_models.UNET_monai_lit import UNET_lit

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/Users/gregory/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-04-15 18:05:41,410 - Created a temporary directory at /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmp6caz56n7
2023-04-15 18:05:41,411 - Writing /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmp6caz56n7/_remote_module_non_scriptable.py


In [3]:
# Initiating Dataset with parameters

# buffer =   -- x,y patchsize for training
# z_start =  --  Offset of slices in the z direction
# z_dim =    -- Number of slices in the z direction. Max value is (64 - z_start)
# validation_rect =  -- rectangle removed for validation set
# shared_height = -- Height to resize all scrolls
# scroll_fragments = -- scrolls to be used 

dataset = Scrolls_Dataset(
                monai= True,
    
                buffer = 32,
                 z_start = 27,
                 z_dim = 8,
                 validation_rect = (1100, 3500, 700, 950),
                shared_height = 8000,
                 downsampling =None,
                 scroll_fragments = [1],#[1,2,3],
                 stage = 'train',
                 shuffle=True,
                 batch_size=64,
                 num_workers =4 ,
                 on_gpu= False,
                          
                         )


In [4]:
# prepeare data, by processng images and loading dataloader

#dataset.prepare_data()

In [5]:
lit_model = UNET_lit(z_dim = dataset.z_dim)

wandb: Currently logged in as: gmarus. Use `wandb login --relogin` to force relogin


In [6]:
trainer = pl.Trainer(
        accelerator='mps',
        benchmark=True,
        max_epochs=20,
        check_val_every_n_epoch= 1,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir='logs/'),
        log_every_n_steps=1,
        
        overfit_batches=0,
        precision=16,
     accumulate_grad_batches=8,
        #strategy="ddp" if devices > 1 else None,
    )

trainer.fit(lit_model, datamodule=dataset)

2023-04-15 18:05:46,012 - Using 16bit Automatic Mixed Precision (AMP)
2023-04-15 18:05:46,024 - GPU available: True (mps), used: True
2023-04-15 18:05:46,024 - TPU available: False, using: 0 TPU cores
2023-04-15 18:05:46,025 - IPU available: False, using: 0 IPUs
2023-04-15 18:05:46,025 - HPU available: False, using: 0 HPUs


/Users/gregory/anaconda3/lib/python3.10/site-packages/lightning_fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
/Users/gregory/anaconda3/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
8it [00:02,  2.78it/s]


2023-04-15 18:05:50,171 - 
  | Name    | Type       | Params
---------------------------------------
0 | metrics | ModuleDict | 0     
1 | model   | UNet       | 1.6 M 
2 | loss    | MaskedLoss | 0     
---------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.507     Total estimated model params size (MB)


/Users/gregory/anaconda3/lib/python3.10/site-packages/lightning_fabric/loggers/csv_logs.py:188: UserWarning: Experiment logs directory logs/lightning_logs/version_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  rank_zero_warn(


Sanity Checking DataLoader 0:   0%|                                                                                                    | 0/2 [00:00<?, ?it/s]

/Users/gregory/anaconda3/lib/python3.10/site-packages/torch/amp/autocast_mode.py:221: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


/Users/gregory/anaconda3/lib/python3.10/site-packages/monai/losses/dice.py:171: UserWarning: MPS: no support for int64 for sum_out_mps, downcasting to a smaller data type (int32/float32). Native support for int64 has been added in macOS 13.3. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1681456119295/work/aten/src/ATen/native/mps/operations/ReduceOps.mm:132.)
  ground_o = torch.sum(target, dim=reduce_axis)
/Users/gregory/anaconda3/lib/python3.10/site-packages/torchmetrics/utilities/checks.py:57: UserWarning: MPS: no support for int64 for min_max, downcasting to a smaller data type (int32/float32). Native support for int64 has been added in macOS 13.3. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1681456119295/work/aten/src/ATen/native/mps/operations/ReduceOps.mm:610.)
  if ignore_index is None and target.min() < 0:


Epoch 0:   2%|█▏                                                                     | 6777/424839 [04:51<4:59:59, 23.23it/s, v_num=0, train/loss_step=0.830]

/Users/gregory/anaconda3/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [7]:
model = UNet(
                spatial_dims=2,
                in_channels=8, #z_dim
                out_channels=1,
                channels=(16, 32, 64, 128, 256),
                strides=(2, 2, 2, 2),
                num_res_units=2,
                dropout=.3,
            )

NameError: name 'UNet' is not defined

In [ ]:
dataloader = iter(dataset.train_dataloader())
train_tensor = None
train_label = None
train_mask = None
for i in range(1):
    # Get image and label from train data -- change number for different ones
    #print(next(dataloader))
    subvolume, inklabel, mask = next(dataloader)
    print('subvolume shape:',subvolume.shape)
    print('inklabel shape:',inklabel.shape)
    print('mask shape', mask.shape)
    train_tensor = subvolume
    train_label = inklabel
    train_mask = mask

In [ ]:
res = model(train_tensor.squeeze(1))
print(res.shape)
loss = monai.losses.DiceLoss(sigmoid=True)
loss = monai.losses.MaskedLoss(loss)
value = loss(res, train_label, train_mask)


In [ ]:
print(value)

In [ ]:
x,y = dataset.pixels[900000]
print(y,x)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(dataset.label_tensors)

BUFFER = 256

patch = patches.Rectangle([y - BUFFER, x - BUFFER], 2 * BUFFER, 2 * BUFFER, linewidth=1, edgecolor='red', facecolor='none')
ax.add_patch(patch)

In [ ]:
x,y = y,x
BUFFER = 256
print(y,x)

In [ ]:
image = dataset.image_tensors[:, y - BUFFER:y + BUFFER , x - BUFFER:x + BUFFER ].view(1, 8, BUFFER * 2 ,BUFFER * 2)





In [ ]:
label =dataset.label_tensors[ y - BUFFER:y + BUFFER  , x - BUFFER:x + BUFFER  ].view(1, BUFFER * 2 ,BUFFER * 2 )










In [ ]:
label.shape

In [ ]:
label.dtype

In [ ]:
fig, ax = plt.subplots()
ax.imshow(dataset.mask)

BUFFER = 256

patch = patches.Rectangle([y - BUFFER, x - BUFFER], 2 * BUFFER, 2 * BUFFER, linewidth=1, edgecolor='red', facecolor='none')
ax.add_patch(patch)

In [ ]:
print(dataset.mask.shape)
print(dataset.label_tensors.shape)

In [ ]:
y,x = x,y
mask = dataset.mask[ y - BUFFER:y + BUFFER  , x - BUFFER:x + BUFFER  ].view(1, BUFFER * 2 ,BUFFER * 2 )
plt.imshow(mask)

In [ ]:
plt.imshow(label)

In [ ]:
label.shape

In [ ]:
image.shape

In [ ]:
fig, ax = plt.subplots()


In [ ]:
dataset.image_tensors.shape

In [ ]:
dataset.label_tensors.shape

In [ ]:
train_tensor.shape

In [ ]:
res = model(train_tensor.squeeze(1))
print(res.shape)
print(train_label.shape)
loss = monai.losses.DiceLoss(sigmoid=True)
loss = monai.losses.MaskedLoss(loss)
value = loss(res, train_label)

In [ ]:
print(res.shape)
print(train_label.shape)

In [ ]:
loss = monai.losses.DiceLoss(sigmoid=True)
    
loss = monai.losses.MaskedLoss(loss)

In [ ]:
value = loss(res, train_label)